In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np

# -Translate- #

def translate_1(model, test_dataloader, tokenizer, max_token_length):
    bleu_scores = []
    model.cpu()
    smoothing = SmoothingFunction()

    for test_batch in test_dataloader:
        test_input = {key: test_batch[key] for key in ('input_ids', 'attention_mask')}
        English_1 = model.generate(
            **test_input,
            max_length=max_token_length,
            num_beams=5,
        )
        labels = np.where(test_batch['labels'] != -100, test_batch['labels'], tokenizer.pad_token_id)

        Gen_sents = tokenizer.batch_decode(test_batch['input_ids'], skip_special_tokens=True)
        references = tokenizer.batch_decode(labels, skip_special_tokens=True)
        preds = tokenizer.batch_decode(English_1, skip_special_tokens=True)

        for s in zip(Gen_sents, references, preds):
            # Calculate BLEU-4 score
            reference_sentence = s[1].split()
            translate_sentence = s[2].split()
            bleu_score = sentence_bleu([reference_sentence], translate_sentence, weights=(0.25, 0.25, 0.25, 0.25),
                                                                             smoothing_function=smoothing.method1)
            bleu_scores.append(bleu_score)

    average_bleu_score = sum(bleu_scores) / len(bleu_scores)
    return average_bleu_score